In [90]:
### To maintain the packages and the the used versions
# !pip install pipreqsnb
!pipreqsnb --force ./

pipreqs  --force ./


INFO: Not scanning for jupyter notebooks.
Please, verify manually the final list of requirements.txt to avoid possible dependency confusions.
INFO: Successfully saved requirements file in ./requirements.txt


In [7]:
# pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


# Loading Data

English to Hindi translation pairs from the MUSE dataset by Meta.

In [26]:
import pandas as pd

In [18]:
train_path = "data/train.txt"
test_path = "data/test.txt"

We are loading the text pairs into a pandas dataframe for easy access. Reading with Pandas was chosen over directly loading from the file as the system was unable to recognize the Devanagri script if loaded directly and was throwing errors. The dataframes are then converted into a dictionaries,

Separate train and test files were downloaded from the github repository and loaded accordingly. The train dataset contains 8704 text pairs and the test set has 2032 pairs. Sample of the dataset is also shown. Each entity of the dataframe is a string. The dataframes are then converted to dictionaries for ease of access.

In [44]:
train_df = pd.read_csv(train_path, sep = '\t', header = None)
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8704 entries, 0 to 8703
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       8704 non-null   object
 1   1       8704 non-null   object
dtypes: object(2)
memory usage: 136.1+ KB


In [46]:
test_df = pd.read_csv(test_path, sep = '\t', header = None)
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2032 entries, 0 to 2031
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       2032 non-null   object
 1   1       2032 non-null   object
dtypes: object(2)
memory usage: 31.9+ KB


In [54]:
train_df.head()

,0,1
0,and,और
1,was,था
2,was,थी
3,for,लिये
4,that,उस


In [96]:
trainPairs = dict(train_df.values)
testPairs = dict(test_df.values)

# Loading pre-trained Fast-text monolingual embeddings

In [9]:
# !pip install fasttext
import fasttext
import fasttext.util

In [13]:
# hindi_embed = fasttext.load_model('wiki.hi.bin')
english_embed = fasttext.load_model('cc.en.300.bin')

ValueError: cc.en.300.bin cannot be opened for loading!